In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv("cleaned_training_data1.csv")
df_test = pd.read_csv("cleaned_test_data1.csv")
df_test.columns

Index(['location_x', 'location_y', 'power_of_shot', 'knockout_match',
       'game_season', 'distance_of_shot', 'is_goal', 'area_of_shot',
       'shot_basics', 'range_of_shot', 'date_of_game', 'home/away',
       'shot_id_number', 'lat/lng', 'match_id', 'remaining_time'],
      dtype='object')

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train["game_season"] = le.fit_transform(df_train.iloc[:,4].values)
df_test["game_season"] = le.transform(df_test.iloc[:,4].values)

one_hot = pd.get_dummies(df_train["area_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("area_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_train["range_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("range_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_train["shot_basics"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("shot_basics" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_train["home/away"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("home/away" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_train["lat/lng"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("lat/lng" , axis=1,inplace=True)
df_train.drop("date_of_game" , axis=1 , inplace=True)
df_train.drop("shot_id_number" ,axis=1 ,inplace=True)
#df_train.drop("match_id" , axis=1 ,inplace=True)

In [4]:
one_hot = pd.get_dummies(df_test["area_of_shot"] , drop_first=True)
df_test = df_test.join(one_hot)
df_test.drop("area_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_test["range_of_shot"] , drop_first=True)
df_test = df_test.join(one_hot)
df_test.drop("range_of_shot" , axis=1, inplace=True)

one_hot = pd.get_dummies(df_test["shot_basics"] , drop_first=True)
df_test = df_test.join(one_hot)
df_test.drop("shot_basics" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_test["home/away"] , drop_first=True)
df_test = df_test.join(one_hot , lsuffix="left" , rsuffix="right")
df_test.drop("home/away" , axis=1 , inplace=True)

one_hot = pd.get_dummies(df_test["lat/lng"] , drop_first=True)
df_test = df_test.join(one_hot)
df_test.drop("lat/lng" , axis=1,inplace=True)
df_test.drop("date_of_game" , axis=1 , inplace=True)
df_test.drop("shot_id_number" ,axis=1 ,inplace=True)
#df_test.drop("match_id" , axis=1 ,inplace=True)

In [5]:
x = df_train.iloc[: , df_train.columns != "is_goal"].values
y = df_train.iloc[: , df_train.columns == "is_goal"].values

In [6]:
X = df_test.iloc[: , df_train.columns != "is_goal"].values
Y = df_test.iloc[: , df_train.columns == "is_goal"].values

# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=2 ,learning_rate=0.05)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test , y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 60.51%


In [8]:
X = sc.transform(X)
y_pred = model.predict_proba(X)

In [9]:
df_test = df_test.iloc[: , df_test.columns != "is_goal"]
importances = pd.DataFrame({"columns":df_test.columns , "importances":model.feature_importances_})
importances.sort_values("importances" ,ascending=False)

,columns,importances
5,distance_of_shot,0.264874
16,Less Than 8 ft.,0.264237
13,24+ ft.,0.093274
7,remaining_time,0.058415
32,MANU @ GSW,0.035811
0,location_x,0.032996
69,MANU vs. GSW,0.032055
33,MANU @ HOU,0.029974
1,location_y,0.027810
4,game_season,0.027702


In [10]:
droplist = []
for i in range(len(importances)):
    if importances["importances"][i] == 0:
        droplist.append(importances["columns"][i])
droplist

df_train1 = df_train.copy()
x = len(df_train1.columns)
for i in range(x):
    flag = 0
    for j in range(len(droplist)):
        if df_train.columns[i] == droplist[j]:
            flag = 1
            break
    if flag == 1:
        df_train1.drop(df_train.columns[i] , axis=1, inplace=True)

df_train1

X = df_train1.iloc[: , df_train1.columns != "is_goal"].values
Y = df_train1.iloc[: , df_train1.columns == "is_goal"].values

# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from xgboost import XGBClassifier
model = XGBClassifier(max_depth=2 ,learning_rate=0.05)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test , y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

len(model.feature_importances_)

temp = df_train1.iloc[: ,  df_train1.columns != "is_goal"]
temp

importances = pd.DataFrame({"columns":temp.columns , "importances":model.feature_importances_})
importances.sort_values("importances" ,ascending=False)

c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 60.51%


,columns,importances
4,distance_of_shot,0.264874
8,Less Than 8 ft.,0.264238
7,24+ ft.,0.093274
6,remaining_time,0.058415
10,MANU @ GSW,0.035812
0,location_x,0.032996
15,MANU vs. GSW,0.032055
11,MANU @ HOU,0.029974
1,location_y,0.027810
3,game_season,0.027702


In [11]:
droplist = []
for i in range(len(importances)):
    if importances["importances"][i] == 0:
        droplist.append(importances["columns"][i])

df_train2 = df_train1.copy()
x = len(df_train1.columns)
for i in range(x):
    flag = 0
    for j in range(len(droplist)):
        if df_train1.columns[i] == droplist[j]:
            flag = 1
            break
    if flag == 1:
        df_train2.drop(df_train1.columns[i] , axis=1, inplace=True)

df_train2

X = df_train2.iloc[: , df_train2.columns != "is_goal"].values
Y = df_train2.iloc[: , df_train2.columns == "is_goal"].values

# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from xgboost import XGBClassifier
model = XGBClassifier(max_depth=2 ,learning_rate=0.05)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test , y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

len(model.feature_importances_)

temp = df_train2.iloc[: ,  df_train2.columns != "is_goal"]
temp

importances = pd.DataFrame({"columns":temp.columns , "importances":model.feature_importances_})
importances.sort_values("importances" ,ascending=False)

c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\bharat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 60.51%


,columns,importances
4,distance_of_shot,0.264874
8,Less Than 8 ft.,0.264238
7,24+ ft.,0.093274
6,remaining_time,0.058415
10,MANU @ GSW,0.035812
0,location_x,0.032996
15,MANU vs. GSW,0.032055
11,MANU @ HOU,0.029974
1,location_y,0.027810
3,game_season,0.027702
